In [4]:
from tqdm import tqdm
import csv
import numpy as np
from src.config import TLCPaths
import json 

In [5]:
with open(TLCPaths.project_data_path.joinpath("mention_and_cui.json"), "r") as f:
    data = json.load(f)
    X = data["X"]
    Y = data["Y"]

In [6]:
import sqlite3

db_path = "/home/tim/PycharmProjects/py-umls/databases/umls.db"
db = sqlite3.connect(db_path)
cur = db.cursor()

In [7]:
for res in cur.execute("SELECT * FROM descriptions WHERE LAT = 'ENG' LIMIT 5"):
    print(res)

('C0000005', 'ENG', 'MSH', 'PEP', '(131)I-Macroaggregated Albumin', 'T116|T121|T130')
('C0000039', 'ENG', 'MTH', 'PN', '1,2-dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000039', 'ENG', 'MSH', 'MH', '1,2-Dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000039', 'ENG', 'MSH', 'PM', '1,2 Dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000052', 'ENG', 'MTH', 'PN', '1,4-alpha-Glucan Branching Enzyme', 'T116|T126')


In [19]:
cui = "C0005903"
def get_semantic_types(cuis):
    semantic_types = dict()
    for identifier in tqdm(cuis):
        sql = f"SELECT * FROM descriptions WHERE CUI = ? LIMIT 1"
        
        try:
            res = next(cur.execute(sql, (identifier,)))[-1]
            semantic_types[identifier] = res.split("|")
        except StopIteration:
            print('Semantic type for cui ' + "'" + str(identifier) + "'" + ' not found')
    return semantic_types

semantic_types = get_semantic_types(Y)
print(len(semantic_types), semantic_types[Y[0]])

100%|██████████| 381/381 [00:00<00:00, 32274.95it/s]

331 ['T184']


In [24]:
unique_semantic_types = set()
for s_types in semantic_types.values():
    for s_type in s_types:
        unique_semantic_types.add(s_type)
print(unique_semantic_types)

{'T007', 'T060', 'T184', 'T061', 'T048', 'T040', 'T025', 'T059', 'T001', 'T038', 'T037', 'T093', 'T130', 'T109', 'T190', 'T024', 'T123', 'T029', 'T191', 'T047', 'T116', 'T073', 'T020', 'T033', 'T070', 'T121', 'T074', 'T046', 'T131', 'T058', 'T031', 'T091', 'T022', 'T125', 'T023'}


In [32]:
import requests
UMLS_KEY = "43f9234c-4977-45f6-a440-2dda1b43d919"

s_type_names = []
for s_type in unique_semantic_types:
    query_url = f"https://uts-ws.nlm.nih.gov/rest/semantic-network/2022AB/TUI/{s_type}?apiKey={UMLS_KEY}"
    try:
        name = requests.get(query_url).json()["result"]["name"]
    except KeyError:
        name = "unknown"
        print(f"Unknown semantic type: {s_type}")
    s_type_names.append(name)
print(s_type_names)

['Bacterium', 'Diagnostic Procedure', 'Sign or Symptom', 'Therapeutic or Preventive Procedure', 'Mental or Behavioral Dysfunction', 'Organism Function', 'Cell', 'Laboratory Procedure', 'Organism', 'Biologic Function', 'Injury or Poisoning', 'Health Care Related Organization', 'Indicator, Reagent, or Diagnostic Aid', 'Organic Chemical', 'Anatomical Abnormality', 'Tissue', 'Biologically Active Substance', 'Body Location or Region', 'Neoplastic Process', 'Disease or Syndrome', 'Amino Acid, Peptide, or Protein', 'Manufactured Object', 'Acquired Abnormality', 'Finding', 'Natural Phenomenon or Process', 'Pharmacologic Substance', 'Medical Device', 'Pathologic Function', 'Hazardous or Poisonous Substance', 'Health Care Activity', 'Body Substance', 'Biomedical Occupation or Discipline', 'Body System', 'Hormone', 'Body Part, Organ, or Organ Component']
